In [14]:
#Dependencies and api_keys
import requests
import pandas as pd
from pprint import pprint
from config import census_key
import json

In [15]:
# Create list of state codes
state_codes = []
for number in range(1, 10):
    state_codes.append('0{0}'.format(number))
#for number in range(11,52):
#    state_codes.append('{0}'.format(number))
#state_codes[0]

In [19]:
# Loop through state code list, skipping states without data
fail_master = []
year_list = ['2009']
#, '2010','2011', '2012', '2013', '2014', '2015']
for year in year_list:
    for code in state_codes:
        try:
            census_test = requests.get(f'https://api.census.gov/data/{year}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDNET,POP1YR' +
                                   f'&for=county:*&in=state:{code}&key=' +
                                   census_key).json()
            census_df = pd.DataFrame(census_test)
            census_df.columns = census_df.iloc[0]
            census_df = census_df[1:]
            census_df = census_df.rename(columns = {'FULL1_NAME': 'County',
                                                    'FULL2_NAME': 'Origin',
                                                    'MOVEDIN': f'{year}IN',
                                                    'MOVEDNET': f'{year}NET',
                                                    'POP1YR':f'{year}POP1YR'
                                                   })
            if code == '01':
                census_yearly = census_df
                print(year+" "+code+" Success")
            else:
                census_yearly = pd.concat([census_yearly, census_df])
                print(year+" "+code+" Success")
        except:
            fail_master.append(code)
            print(year+" "+code+" FAILED")
    if year == '2009':
        census_master = census_yearly
    else:
        census_yearly = census_yearly[['County', 'Origin', f'{year}IN', f'{year}NET',f'{year}POP1YR']]
        census_master = census_master.merge(census_yearly, on = ['County', 'Origin'])

2009 01 Success
2009 02 Success
2009 03 FAILED
2009 04 Success
2009 05 Success
2009 06 Success
2009 07 FAILED
2009 08 Success
2009 09 Success


In [18]:
fail_master
fail_master
census_master

,County,Origin,2009IN,2009NET,2009NET,state,county
1,"Autauga County, Alabama",Asia,32,None,48877,01,001
2,"Autauga County, Alabama",Caribbean,7,None,48877,01,001
3,"Autauga County, Alabama",Europe,37,None,48877,01,001
4,"Autauga County, Alabama",U.S. Island Areas,18,None,48877,01,001
5,"Autauga County, Alabama",Northern America,11,None,48877,01,001
6,"Autauga County, Alabama",South America,10,None,48877,01,001
7,"Autauga County, Alabama","Baldwin County, Alabama",35,28,48877,01,001
8,"Autauga County, Alabama","Barbour County, Alabama",0,-18,48877,01,001
9,"Autauga County, Alabama","Bibb County, Alabama",38,38,48877,01,001
10,"Autauga County, Alabama","Blount County, Alabama",100,45,48877,01,001


In [13]:
census_master.to_csv('./census_output.csv')